In [3]:
!pip install openai jsonlines bert-score sacrebleu nltk

#!pip install openai
#!pip install jsonlines
#!pip install bert-score
#!pip install sacrebleu
#!pip install nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 3.8 MB/s eta 0:00:00


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import os
from json import JSONEncoder
import json
import requests
import jsonlines
import random
from openai import OpenAI
import openai
from google.colab import drive
from bert_score import score
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import sacrebleu
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk import word_tokenize
from difflib import SequenceMatcher
from bs4 import BeautifulSoup, Tag
import tinycss2

In [5]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
api_key_file_path = '' # Path to the file containing the OpenAI API key

# Read the API key from a file
with open(api_key_file_path, 'r') as file:
    openai_api_key  = file.read().strip()

## Prepare Evaluation Dataset

In [ ]:
import jsonlines

# Path to your JSONL file
jsonl_file_path = '' # Path to the JSONL file

# Load the data from the JSONL file
dataset = []
with jsonlines.open(jsonl_file_path, 'r') as reader:
    for obj in reader:
        dataset.append(obj)

# Verify the data structure by printing the first item
print(f"Total records: {len(dataset)}")
print("Sample data:", dataset[0])

Total records: 263
Sample data: {'messages': [{'role': 'system', 'content': 'Provide structured HTML and CSS for UI components of a web template.'}, {'role': 'user', 'content': 'Create a header of a personal portfolio website with a modern, clean aesthetic. Utilize HTML for logo, navigation bar, and concise intro sections, each with distinct calls-to-action. Employ CSS to emphasize typography, color contrasts, and hover interactions, focusing on readability and intuitive layout for seamless navigation while aligning with UI/UX best practices.'}, {'role': 'assistant', 'content': '<html>\n<head>\n<style>\n.icon {\n  \tposition: absolute;\n  \ttop: 42px;\n  \tleft: 0px;\n  \twidth: 36.4px;\n  \theight: 34px;\n  \toverflow: hidden;\n}\n.devstack {\n  \tcolor: #3f3a64;\n}\n.v-devstack {\n  \tposition: absolute;\n  \ttop: 0px;\n  \tleft: 36.44px;\n  \tline-height: 51px;\n  \ttext-transform: capitalize;\n}\n.link {\n  \tposition: absolute;\n  \ttop: calc(50% - 25.51px);\n  \tleft: 0px;\n  \tw

## Splitting data

In [ ]:
# Set a random seed for reproducibility
random.seed(42)

# Define split ratios
train_ratio = 0.8
val_ratio = 0.1
test_ratio = 0.1

# Shuffle the dataset
random.shuffle(dataset)

# Calculate split indices
n_total = len(dataset)
n_train = int(n_total * train_ratio)
n_val = int(n_total * val_ratio)

# Split the dataset
train_data = dataset[:n_train]
val_data = dataset[n_train:n_train + n_val]
test_data = dataset[n_train + n_val:]

# Verify the sizes of each split
print(f"Training set size: {len(train_data)}")
print(f"Validation set size: {len(val_data)}")
print(f"Test set size: {len(test_data)}")

Training set size: 210
Validation set size: 26
Test set size: 27


In [ ]:
# Limit the test set to only 5 samples to avoid hitting rate limits
test_data = dataset[int(0.9 * len(dataset)):][:5]

In [ ]:
# Define the exact prompts you want to filter for
desired_prompts = [
    "Create a header for a portfolio website with a focus on personal branding using a minimalist and modern look. Integrate HTML components like a headline, description, and call-to-action buttons, along with social media links. Use CSS for cohesive typography, vibrant color highlights, and a user-centered layout. Aim for visual hierarchy, intuitive navigation, and a responsive design to enhance user engagement.",

    "Create a header of a Professional Portfolio Website focusing on a creative and vibrant layout. Utilize HTML for structured sections like About Me, Services, and buttons such as Hire Me and Download CV. Style with CSS to emphasize a modern UI, incorporating color gradients, rounded buttons, and clear typography, enhancing readability and user interaction while ensuring visual consistency.",

    "Create a header of a fashion e-commerce website, featuring a clean, modern look. Include an announcement bar, logo, navigation menu for New Arrivals, Swimwear, Collections, About Us, and Demos, a search icon, account link, and cart icon. Use responsive design, bold typography, intuitive layout, and strategically placed call-to-action buttons to enhance user experience and engagement.",

    "Create a header of a Portfolio Website for a Freelance UI/UX Designer with a modern and visually dynamic look. Implement hero imagery, impactful typography, clear call to action buttons, and a well-structured introduction using HTML and CSS. Emphasize visual hierarchy, white space, and responsive design, adhering to UI/UX principles to enhance user engagement and accessibility.",

    "Create a header of a Creative Agency Website, emphasizing a modern and minimalist design. Utilize HTML and CSS to structure a responsive navigation bar with bold branding, clear call-to-actions, and intuitive menu links. Implement visually striking elements, balanced whitespace, and consistent typography to enhance user engagement while adhering to UI/UX best practices for accessibility and seamless interaction.",

    "Create a header of an eCommerce website under the 'Products' category, featuring a modern and clean look with navigation links (Home, Recent Products, Brands, Contact, About), a search bar, cart icon, and branding. Include all HTML and CSS components to ensure responsiveness and intuitive UI/UX, optimizing for user engagement and seamless shopping experience."
]

# Use a set to keep track of unique prompts and filter out duplicates
unique_prompts = set()
unique_test_data = []

for item in test_data:
    prompt = item['messages'][1]['content']
    if prompt not in unique_prompts:
        unique_prompts.add(prompt)
        unique_test_data.append(item)

# Update test_data with the unique samples only
test_data = unique_test_data

# Print the number of selected samples and their prompts to verify
print(f"Filtered test set size (unique prompts): {len(test_data)}")
for i, item in enumerate(test_data, 1):
    print(f"Sample {i} Prompt: {item['messages'][1]['content']}")

Filtered test set size (unique prompts): 6
Sample 1 Prompt: Create a header for a portfolio website with a focus on personal branding using a minimalist and modern look. Integrate HTML components like a headline, description, and call-to-action buttons, along with social media links. Use CSS for cohesive typography, vibrant color highlights, and a user-centered layout. Aim for visual hierarchy, intuitive navigation, and a responsive design to enhance user engagement.
Sample 2 Prompt: Create a header of a Creative Agency Website, emphasizing a modern and minimalist design. Utilize HTML and CSS to structure a responsive navigation bar with bold branding, clear call-to-actions, and intuitive menu links. Implement visually striking elements, balanced whitespace, and consistent typography to enhance user engagement while adhering to UI/UX best practices for accessibility and seamless interaction.
Sample 3 Prompt: Create a header of a Portfolio Website for a Freelance UI/UX Designer with a m

### Calculating BERTScore

In [ ]:
# Extract reference responses (ground truth) from the test data
reference_responses = [item['messages'][2]['content'] for item in test_data]

# Set up headers for OpenAI API
headers = {
    "Authorization": f"Bearer {openai_api_key}",
    "Content-Type": "application/json"
}

# Fine-tuned model ID
fine_tuned_model_id = ""  # Replace with your actual fine-tuned model ID

# Generate model predictions for each item in the test data
model_predictions = []
for item in test_data:
    # Prepare the sample conversation
    sample_conversation = [
        {"role": "system", "content": "Provide structured HTML and CSS for UI components of a web template."},
        {"role": "user", "content": item['messages'][1]['content']}
    ]

    # Send the request to the OpenAI API
    response = requests.post(
        "https://api.openai.com/v1/chat/completions",
        headers=headers,
        json={
            "model": fine_tuned_model_id,
            "messages": sample_conversation
        }
    )

    # Handle the response
    if response.status_code == 200:
        completion = response.json()["choices"][0]["message"]["content"]
        model_predictions.append(completion.strip())
    else:
        print("Error:", response.status_code, response.text)
        model_predictions.append("")  # Append empty string if there's an error

# Calculate BERTScore
P, R, F1 = score(model_predictions, reference_responses, lang="en", rescale_with_baseline=True)

# Display the average F1 score as the similarity metric
average_f1_score = F1.mean().item()
print(f"Average BERTScore F1 for fine-tuned GPT-4 model (5 samples): {average_f1_score:.4f}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Average BERTScore F1 for fine-tuned GPT-4 model (5 samples): 0.5961


### CodeBLEU Evaluation (Does Not Works on HTML/CSS)

In [ ]:
# Clone the repository and navigate to the CodeBLEU directory
!git clone https://github.com/microsoft/CodeXGLUE.git
%cd CodeXGLUE/Code-Code/code-to-code-trans/evaluator/CodeBLEU

Cloning into 'CodeXGLUE'...
remote: Enumerating objects: 3373, done.
remote: Counting objects: 100% (3372/3372), done.
remote: Compressing objects: 100% (1534/1534), done.
remote: Total 3373 (delta 1748), reused 3326 (delta 1733), pack-reused 1 (from 1)
Receiving objects: 100% (3373/3373), 213.15 MiB | 13.06 MiB/s, done.
Resolving deltas: 100% (1748/1748), done.
Updating files: 100% (400/400), done.
/content/CodeXGLUE/Code-Code/code-to-code-trans/evaluator/CodeBLEU
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


In [ ]:
!pip install tree_sitter nltk sacrebleu requests

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.6/566.6 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 7.1 MB/s eta 0:00:00


In [ ]:
# Set up headers for OpenAI API
headers = {
    "Authorization": f"Bearer {openai_api_key}",  # Replace with your OpenAI API key
    "Content-Type": "application/json"
}

# Fine-tuned model ID
fine_tuned_model_id = ""  # Replace with your actual fine-tuned model ID

# Generate predictions for each item in the test set
model_predictions = []
for item in test_data:
    # Prepare the sample conversation for each user prompt in test data
    sample_conversation = [
        {"role": "system", "content": "Provide structured HTML and CSS for UI components of a web template."},
        {"role": "user", "content": item['messages'][1]['content']}
    ]

    # Call the model and get the response
    response = requests.post(
        "https://api.openai.com/v1/chat/completions",
        headers=headers,
        json={
            "model": fine_tuned_model_id,
            "messages": sample_conversation
        }
    )

    # Handle the response
    if response.status_code == 200:
        completion = response.json()["choices"][0]["message"]["content"]
        model_predictions.append(completion.strip())
    else:
        print("Error:", response.status_code, response.text)
        model_predictions.append("")  # Append empty string if there's an error

# Define your Google Drive paths
predictions_path = '/content/drive/MyDrive/Sem 3/DATA 298 B/Data/predictions.txt'
references_path = '/content/drive/MyDrive/Sem 3/DATA 298 B/Data/references.txt'

# Save model predictions to your specified path in Google Drive
with open(predictions_path, 'w') as f:
    for pred in model_predictions:
        f.write(pred.strip() + '\n')

# Save reference responses (ground truth) to your specified path in Google Drive
with open(references_path, 'w') as f:
    for ref in reference_responses:
        f.write(ref.strip() + '\n')

print("Files saved to Google Drive.")

Files saved to Google Drive.


In [ ]:
!python calc_code_bleu.py --refs /content/drive/MyDrive/Sem\ 3/DATA\ 298\ B/Data/references.txt --hyp /content/drive/MyDrive/Sem\ 3/DATA\ 298\ B/Data/predictions.txt --lang js --params 0.25,0.25,0.25,0.25

Traceback (most recent call last):
  File "/content/CodeXGLUE/Code-Code/code-to-code-trans/evaluator/CodeBLEU/calc_code_bleu.py", line 33, in <module>
    assert len(hypothesis) == len(pre_references[i])
AssertionError


In [ ]:
# Check line counts of predictions and references
with open('/Data/predictions.txt', 'r') as pred_file:
    predictions_lines = pred_file.readlines()

with open('Data/references.txt', 'r') as ref_file:
    references_lines = ref_file.readlines()

print(f"Predictions lines: {len(predictions_lines)}")
print(f"References lines: {len(references_lines)}")

if len(predictions_lines) != len(references_lines):
    print("Mismatch in line counts. Please ensure both files have the same number of lines.")
else:
    print("Line counts match. Ready to run CodeBLEU.")

Predictions lines: 1086
References lines: 4304
Mismatch in line counts. Please ensure both files have the same number of lines.


## Evaluating CHRF (CHaRacter-level F-score)

In [ ]:
import requests
from sacrebleu.metrics import CHRF
import random

# Set up headers for OpenAI API
headers = {
    "Authorization": f"Bearer {openai_api_key}",  # Replace with your OpenAI API key
    "Content-Type": "application/json"
}

# Fine-tuned model ID
fine_tuned_model_id = ""  # Replace with your actual fine-tuned model ID

# Placeholder lists for model predictions and reference responses
model_predictions = []
reference_responses = [item['messages'][2]['content'] for item in test_data]

# Ensure equal length by limiting the number of calls if necessary
sample_size = min(5, len(reference_responses))  # Set to 5 samples or less if references are fewer
test_data_sample = random.sample(test_data, sample_size)

for item in test_data_sample:
    # Prepare the sample conversation for each user prompt in test data
    sample_conversation = [
        {"role": "system", "content": "Provide structured HTML and CSS for UI components of a web template."},
        {"role": "user", "content": item['messages'][1]['content']}
    ]

    # Call the model and get the response
    response = requests.post(
        "https://api.openai.com/v1/chat/completions",
        headers=headers,
        json={
            "model": fine_tuned_model_id,
            "messages": sample_conversation
        }
    )

    # Handle the response
    if response.status_code == 200:
        completion = response.json()["choices"][0]["message"]["content"]
        if completion.strip():  # Check if completion is non-empty
            model_predictions.append(completion.strip())
    else:
        print("Error:", response.status_code, response.text)
        model_predictions.append("")  # Append empty string if there's an error

# Filter empty predictions
model_predictions = [pred for pred in model_predictions if pred]
reference_responses = reference_responses[:len(model_predictions)]

# Verify lengths for CHRF calculation
assert len(model_predictions) == len(reference_responses), "The lengths of predictions and references do not match."

# Calculate CHRF score
chrf = CHRF()
score = chrf.corpus_score(model_predictions, [reference_responses])
print(f"CHRF Score: {score.score:.4f}")

CHRF Score: 53.7893


### Evaluating BLEU

In [ ]:
# Define a smoothing function to avoid BLEU score of 0 for short texts
smoothie = SmoothingFunction().method4

# Calculate BLEU scores for each prediction-reference pair
bleu_scores = []
for pred, ref in zip(model_predictions, reference_responses):
    # Tokenize both prediction and reference as lists of words
    pred_tokens = pred.split()
    ref_tokens = ref.split()

    # Calculate BLEU score for this pair with smoothing
    score = sentence_bleu([ref_tokens], pred_tokens, smoothing_function=smoothie)
    bleu_scores.append(score)

# Calculate the average BLEU score across all samples
average_bleu_score = sum(bleu_scores) / len(bleu_scores)
print(f"Average BLEU Score: {average_bleu_score:.4f}")


Average BLEU Score: 0.1850


### Evaluating Ruby

##### Ruby STS

In [ ]:
nltk.download('punkt')

# Set up headers for OpenAI API
headers = {
    "Authorization": f"Bearer {openai_api_key}",  # Replace with your OpenAI API key
    "Content-Type": "application/json"
}

# Fine-tuned model ID
fine_tuned_model_id = ""  # Replace with your actual fine-tuned model ID

# Initialize lists for storing generated and reference data
model_predictions = []
reference_responses = [item['messages'][2]['content'] for item in test_data]  # Extract reference output from test data

# Generate predictions for each item in the test set
for item in test_data:
    # Prepare the sample conversation
    sample_conversation = [
        {"role": "system", "content": "Provide structured HTML and CSS for UI components of a web template."},
        {"role": "user", "content": item['messages'][1]['content']}
    ]

    # Call the model and get the response
    response = requests.post(
        "https://api.openai.com/v1/chat/completions",
        headers=headers,
        json={
            "model": fine_tuned_model_id,
            "messages": sample_conversation
        }
    )

    # Handle the response
    if response.status_code == 200:
        completion = response.json()["choices"][0]["message"]["content"]
        model_predictions.append(completion.strip())
    else:
        print("Error:", response.status_code, response.text)
        model_predictions.append("")  # Append empty string if there's an error

# Function to calculate RUBY's STS approximation using weighted string edit similarity
def weighted_string_edit_similarity(generated, reference):
    """
    Calculate similarity between tokenized HTML/CSS strings using SequenceMatcher.
    """
    generated_tokens = word_tokenize(generated)
    reference_tokens = word_tokenize(reference)
    matcher = SequenceMatcher(None, generated_tokens, reference_tokens)
    similarity_ratio = matcher.ratio()
    return similarity_ratio

# Calculate similarity scores for all pairs of generated and reference data
similarity_scores = []
for gen, ref in zip(model_predictions, reference_responses):
    score = weighted_string_edit_similarity(gen, ref)
    similarity_scores.append(score)

# Display average similarity score
average_similarity = sum(similarity_scores) / len(similarity_scores)
print(f"Average RUBY STS Similarity Score (approximation): {average_similarity:.2f}")

# Display individual scores (optional)
for i, score in enumerate(similarity_scores, 1):
    print(f"Sample {i}: Similarity Score = {score:.2f}")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Average RUBY STS Similarity Score (approximation): 0.20
Sample 1: Similarity Score = 0.17
Sample 2: Similarity Score = 0.22
Sample 3: Similarity Score = 0.19
Sample 4: Similarity Score = 0.19
Sample 5: Similarity Score = 0.23
Sample 6: Similarity Score = 0.21


##### Ruby AST + STS

In [ ]:
nltk.download('punkt')

# Set up headers for OpenAI API
headers = {
    "Authorization": f"Bearer {openai_api_key}",  # Replace with your OpenAI API key
    "Content-Type": "application/json"
}

# Fine-tuned model ID
fine_tuned_model_id = "ft:gpt-4o-2024-08-06:personal::ANAuc42x"  # Replace with your actual fine-tuned model ID

# Function to calculate STS similarity
def calculate_sts_similarity(generated_code, reference_code):
    generated_tokens = word_tokenize(generated_code)
    reference_tokens = word_tokenize(reference_code)
    matcher = SequenceMatcher(None, generated_tokens, reference_tokens)
    return matcher.ratio() * 100  # Return as percentage

# AST-based similarity for HTML using BeautifulSoup
def calculate_ast_similarity_html(generated_code, reference_code):
    generated_tree = BeautifulSoup(generated_code, "html.parser")
    reference_tree = BeautifulSoup(reference_code, "html.parser")

    def compare_trees(node1, node2):
        if not isinstance(node1, Tag) or not isinstance(node2, Tag):
            return 0  # Ignore non-tag elements
        if node1.name != node2.name:
            return 0  # Different tags

        score = 1  # Start score for matching tag names
        if node1.attrs == node2.attrs:
            score += 1  # Matching attributes increase score

        # Recursively compare children if they exist
        children1 = [child for child in node1.children if isinstance(child, Tag)]
        children2 = [child for child in node2.children if isinstance(child, Tag)]

        for child1, child2 in zip(children1, children2):
            score += compare_trees(child1, child2)
        return score

    try:
        ast_html_score = compare_trees(generated_tree, reference_tree) / max(len(generated_tree.find_all()), len(reference_tree.find_all()))
    except Exception as e:
        print(f"HTML AST comparison failed: {e}")
        ast_html_score = 0
    return ast_html_score

# AST-based similarity for CSS using TinyCSS2
def calculate_ast_similarity_css(generated_code, reference_code):
    generated_rules = tinycss2.parse_stylesheet(generated_code)
    reference_rules = tinycss2.parse_stylesheet(reference_code)

    def compare_css_rules(rule1, rule2):
        if rule1.type != rule2.type:
            return 0
        score = 1  # Matching rule type
        if hasattr(rule1, 'content') and hasattr(rule2, 'content') and rule1.content == rule2.content:
            score += 1  # Matching content
        return score

    score = 0
    try:
        for rule1, rule2 in zip(generated_rules, reference_rules):
            score += compare_css_rules(rule1, rule2)
        ast_css_score = score / max(len(generated_rules), len(reference_rules))
    except Exception as e:
        print(f"CSS AST comparison failed: {e}")
        ast_css_score = 0
    return ast_css_score

# Wrapper function for RUBY metric with AST and STS options
def ruby_metric(generated_code, reference_code):
    try:
        ast_html_score = calculate_ast_similarity_html(generated_code, reference_code)
        ast_css_score = calculate_ast_similarity_css(generated_code, reference_code)
        ast_score = (ast_html_score + ast_css_score) / 2
    except Exception as e:
        print(f"AST comparison failed: {e}")
        ast_score = 0

    sts_score = calculate_sts_similarity(generated_code, reference_code)
    combined_score = (ast_score + sts_score) / 2
    return ast_score, sts_score, combined_score

# Function to evaluate RUBY on a sample of the test dataset
def evaluate_ruby_sample(test_data, sample_size=5):
    sample_data = random.sample(test_data, sample_size)

    ast_scores = []
    sts_scores = []
    combined_scores = []

    for item in sample_data:
        prompt = item['messages'][1]['content']
        reference_code = item['messages'][2]['content']

        sample_conversation = [
            {"role": "system", "content": "Provide structured HTML and CSS for UI components of a web template."},
            {"role": "user", "content": prompt}
        ]

        response = requests.post(
            "https://api.openai.com/v1/chat/completions",
            headers=headers,
            json={
                "model": fine_tuned_model_id,
                "messages": sample_conversation
            }
        )

        if response.status_code == 200:
            generated_code = response.json()["choices"][0]["message"]["content"].strip()
        else:
            print("Error:", response.status_code, response.text)
            generated_code = ""

        # Calculate RUBY metric scores
        ast_score, sts_score, combined_score = ruby_metric(generated_code, reference_code)
        ast_scores.append(ast_score)
        sts_scores.append(sts_score)
        combined_scores.append(combined_score)

        # print(f"Prompt: {prompt}")
        # print(f"Generated Code: {generated_code}")
        # print(f"Reference Code: {reference_code}")
        print(f"AST Score (Average HTML & CSS): {ast_score}")
        print(f"STS Score: {sts_score}")
        print(f"Combined Score: {combined_score}")
        print("-" * 50)

    avg_ast_score = sum(ast_scores) / len(ast_scores) if ast_scores else 0
    avg_sts_score = sum(sts_scores) / len(sts_scores) if sts_scores else 0
    avg_combined_score = sum(combined_scores) / len(combined_scores) if combined_scores else 0

    print(f"Average AST Score: {avg_ast_score}")
    print(f"Average STS Score: {avg_sts_score}")
    print(f"Average Combined Score: {avg_combined_score}")

    return avg_ast_score, avg_sts_score, avg_combined_score

# Run RUBY evaluation on a sample of the test dataset
avg_ast_score, avg_sts_score, avg_combined_score = evaluate_ruby_sample(test_data, sample_size=5)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


AST Score (Average HTML & CSS): 0.46851441241685143
STS Score: 21.428571428571427
Combined Score: 10.94854292049414
--------------------------------------------------
AST Score (Average HTML & CSS): 0.7903225806451613
STS Score: 28.645383951682486
Combined Score: 14.717853266163823
--------------------------------------------------
AST Score (Average HTML & CSS): 0.7191308956014839
STS Score: 17.01323251417769
Combined Score: 8.866181704889588
--------------------------------------------------
AST Score (Average HTML & CSS): 0.2922675026123302
STS Score: 13.173340289552629
Combined Score: 6.732803896082479
--------------------------------------------------
AST Score (Average HTML & CSS): 0.38387607417458164
STS Score: 16.614664586583462
Combined Score: 8.499270330379021
--------------------------------------------------
Average AST Score: 0.5308222930900817
Average STS Score: 19.375038554113537
Average Combined Score: 9.95293042360181


In [ ]:
nltk.download('punkt')

# Set up headers for OpenAI API
headers = {
    "Authorization": f"Bearer {openai_api_key}",  # Replace with your OpenAI API key
    "Content-Type": "application/json"
}

# Fine-tuned model ID
fine_tuned_model_id = "ft:gpt-4o-2024-08-06:personal::ANAuc42x"  # Replace with your actual fine-tuned model ID

# Function to calculate STS similarity
def calculate_sts_similarity(generated_code, reference_code):
    generated_tokens = word_tokenize(generated_code)
    reference_tokens = word_tokenize(reference_code)
    matcher = SequenceMatcher(None, generated_tokens, reference_tokens)
    return matcher.ratio()  # Return as a ratio between 0 and 1

# AST-based similarity for HTML using BeautifulSoup
def calculate_ast_similarity_html(generated_code, reference_code):
    generated_tree = BeautifulSoup(generated_code, "html.parser")
    reference_tree = BeautifulSoup(reference_code, "html.parser")

    def compare_trees(node1, node2):
        if not isinstance(node1, Tag) or not isinstance(node2, Tag):
            return 0  # Ignore non-tag elements
        if node1.name != node2.name:
            return 0  # Different tags

        score = 1  # Start score for matching tag names
        if node1.attrs == node2.attrs:
            score += 1  # Matching attributes increase score

        # Recursively compare children if they exist
        children1 = [child for child in node1.children if isinstance(child, Tag)]
        children2 = [child for child in node2.children if isinstance(child, Tag)]

        for child1, child2 in zip(children1, children2):
            score += compare_trees(child1, child2)
        return score

    try:
        ast_html_score = compare_trees(generated_tree, reference_tree) / max(len(generated_tree.find_all()), len(reference_tree.find_all()))
    except Exception as e:
        print(f"HTML AST comparison failed: {e}")
        ast_html_score = 0
    return ast_html_score

# AST-based similarity for CSS using TinyCSS2
def calculate_ast_similarity_css(generated_code, reference_code):
    generated_rules = tinycss2.parse_stylesheet(generated_code)
    reference_rules = tinycss2.parse_stylesheet(reference_code)

    def compare_css_rules(rule1, rule2):
        if rule1.type != rule2.type:
            return 0
        score = 1  # Matching rule type
        if hasattr(rule1, 'content') and hasattr(rule2, 'content') and rule1.content == rule2.content:
            score += 1  # Matching content
        return score

    score = 0
    try:
        for rule1, rule2 in zip(generated_rules, reference_rules):
            score += compare_css_rules(rule1, rule2)
        ast_css_score = score / max(len(generated_rules), len(reference_rules))
    except Exception as e:
        print(f"CSS AST comparison failed: {e}")
        ast_css_score = 0
    return ast_css_score

# Wrapper function for RUBY metric with AST and STS options
def ruby_metric(generated_code, reference_code):
    try:
        ast_html_score = calculate_ast_similarity_html(generated_code, reference_code)
        ast_css_score = calculate_ast_similarity_css(generated_code, reference_code)
        ast_score = (ast_html_score + ast_css_score) / 2
    except Exception as e:
        print(f"AST comparison failed: {e}")
        ast_score = 0

    sts_score = calculate_sts_similarity(generated_code, reference_code)
    combined_score = (ast_score + sts_score) / 2
    return ast_score, sts_score, combined_score

# Function to evaluate RUBY on a sample of the test dataset
def evaluate_ruby_sample(test_data, sample_size=5):
    sample_data = random.sample(test_data, sample_size)

    ast_scores = []
    sts_scores = []
    combined_scores = []

    for item in sample_data:
        prompt = item['messages'][1]['content']
        reference_code = item['messages'][2]['content']

        sample_conversation = [
            {"role": "system", "content": "Provide structured HTML and CSS for UI components of a web template."},
            {"role": "user", "content": prompt}
        ]

        response = requests.post(
            "https://api.openai.com/v1/chat/completions",
            headers=headers,
            json={
                "model": fine_tuned_model_id,
                "messages": sample_conversation
            }
        )

        if response.status_code == 200:
            generated_code = response.json()["choices"][0]["message"]["content"].strip()
        else:
            print("Error:", response.status_code, response.text)
            generated_code = ""

        # Calculate RUBY metric scores
        ast_score, sts_score, combined_score = ruby_metric(generated_code, reference_code)
        ast_scores.append(ast_score)
        sts_scores.append(sts_score)
        combined_scores.append(combined_score)

        print(f"AST Score (Average HTML & CSS): {ast_score}")
        print(f"STS Score: {sts_score}")
        print(f"Combined Score: {combined_score}")
        print("-" * 50)

    avg_ast_score = sum(ast_scores) / len(ast_scores) if ast_scores else 0
    avg_sts_score = sum(sts_scores) / len(sts_scores) if sts_scores else 0
    avg_combined_score = sum(combined_scores) / len(combined_scores) if combined_scores else 0

    print(f"Average AST Score: {avg_ast_score}")
    print(f"Average STS Score: {avg_sts_score}")
    print(f"Average Combined Score: {avg_combined_score}")

    return avg_ast_score, avg_sts_score, avg_combined_score

# Run RUBY evaluation on a sample of the test dataset
avg_ast_score, avg_sts_score, avg_combined_score = evaluate_ruby_sample(test_data, sample_size=5)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


AST Score (Average HTML & CSS): 0.1040838520963024
STS Score: 0.010445122931062189
Combined Score: 0.057264487513682294
--------------------------------------------------
AST Score (Average HTML & CSS): 0.4369645042839657
STS Score: 0.21451104100946372
Combined Score: 0.3257377726467147
--------------------------------------------------
AST Score (Average HTML & CSS): 0.92
STS Score: 0.34619188921859545
Combined Score: 0.6330959446092977
--------------------------------------------------
AST Score (Average HTML & CSS): 0.6295793758480326
STS Score: 0.14385150812064965
Combined Score: 0.3867154419843411
--------------------------------------------------
AST Score (Average HTML & CSS): 0.3747252747252747
STS Score: 0.11246943765281174
Combined Score: 0.24359735618904324
--------------------------------------------------
Average AST Score: 0.49307060139071507
Average STS Score: 0.16549379978651652
Average Combined Score: 0.3292822005886158


##### Ruby TRS

In [ ]:
nltk.download('punkt')

# Set up headers for OpenAI API
headers = {
    "Authorization": f"Bearer {openai_api_key}",  # Replace with your OpenAI API key
    "Content-Type": "application/json"
}

# Fine-tuned model ID
fine_tuned_model_id = "ft:gpt-4o-2024-08-06:personal::ANAuc42x"  # Replace with your actual fine-tuned model ID

# Function to calculate STS similarity
def calculate_sts_similarity(generated_code, reference_code):
    generated_tokens = word_tokenize(generated_code)
    reference_tokens = word_tokenize(reference_code)
    matcher = SequenceMatcher(None, generated_tokens, reference_tokens)
    return matcher.ratio()  # Return as a ratio between 0 and 1

# AST-based similarity for HTML using BeautifulSoup
def calculate_ast_similarity_html(generated_code, reference_code):
    generated_tree = BeautifulSoup(generated_code, "html.parser")
    reference_tree = BeautifulSoup(reference_code, "html.parser")

    def compare_trees(node1, node2):
        if not isinstance(node1, Tag) or not isinstance(node2, Tag):
            return 0  # Ignore non-tag elements
        if node1.name != node2.name:
            return 0  # Different tags

        score = 1  # Start score for matching tag names
        if node1.attrs == node2.attrs:
            score += 1  # Matching attributes increase score

        # Recursively compare children if they exist
        children1 = [child for child in node1.children if isinstance(child, Tag)]
        children2 = [child for child in node2.children if isinstance(child, Tag)]

        for child1, child2 in zip(children1, children2):
            score += compare_trees(child1, child2)
        return score

    try:
        ast_html_score = compare_trees(generated_tree, reference_tree) / max(len(generated_tree.find_all()), len(reference_tree.find_all()))
    except Exception as e:
        print(f"HTML AST comparison failed: {e}")
        ast_html_score = 0
    return ast_html_score

# AST-based similarity for CSS using TinyCSS2
def calculate_ast_similarity_css(generated_code, reference_code):
    generated_rules = tinycss2.parse_stylesheet(generated_code)
    reference_rules = tinycss2.parse_stylesheet(reference_code)

    def compare_css_rules(rule1, rule2):
        if rule1.type != rule2.type:
            return 0
        score = 1  # Matching rule type
        if hasattr(rule1, 'content') and hasattr(rule2, 'content') and rule1.content == rule2.content:
            score += 1  # Matching content
        return score

    score = 0
    try:
        for rule1, rule2 in zip(generated_rules, reference_rules):
            score += compare_css_rules(rule1, rule2)
        ast_css_score = score / max(len(generated_rules), len(reference_rules))
    except Exception as e:
        print(f"CSS AST comparison failed: {e}")
        ast_css_score = 0
    return ast_css_score

# TRS-based similarity for HTML and CSS
def calculate_trs_similarity(generated_code, reference_code):
    generated_tokens = word_tokenize(generated_code)
    reference_tokens = word_tokenize(reference_code)

    matcher = SequenceMatcher(None, generated_tokens, reference_tokens)
    trs_similarity = matcher.ratio()  # TRS as token-level similarity ratio
    return trs_similarity

# Wrapper function for RUBY metric with AST, STS, and TRS options
def ruby_metric(generated_code, reference_code):
    try:
        ast_html_score = calculate_ast_similarity_html(generated_code, reference_code)
        ast_css_score = calculate_ast_similarity_css(generated_code, reference_code)
        ast_score = (ast_html_score + ast_css_score) / 2
    except Exception as e:
        print(f"AST comparison failed: {e}")
        ast_score = 0

    sts_score = calculate_sts_similarity(generated_code, reference_code)
    trs_score = calculate_trs_similarity(generated_code, reference_code)
    combined_score = (ast_score + sts_score + trs_score) / 3
    return ast_score, sts_score, trs_score, combined_score

# Function to evaluate RUBY on a sample of the test dataset
def evaluate_ruby_sample(test_data, sample_size=5):
    sample_data = random.sample(test_data, sample_size)

    ast_scores = []
    sts_scores = []
    trs_scores = []
    combined_scores = []

    for item in sample_data:
        prompt = item['messages'][1]['content']
        reference_code = item['messages'][2]['content']

        sample_conversation = [
            {"role": "system", "content": "Provide structured HTML and CSS for UI components of a web template."},
            {"role": "user", "content": prompt}
        ]

        response = requests.post(
            "https://api.openai.com/v1/chat/completions",
            headers=headers,
            json={
                "model": fine_tuned_model_id,
                "messages": sample_conversation
            }
        )

        if response.status_code == 200:
            generated_code = response.json()["choices"][0]["message"]["content"].strip()
        else:
            print("Error:", response.status_code, response.text)
            generated_code = ""

        # Calculate RUBY metric scores
        ast_score, sts_score, trs_score, combined_score = ruby_metric(generated_code, reference_code)
        ast_scores.append(ast_score)
        sts_scores.append(sts_score)
        trs_scores.append(trs_score)
        combined_scores.append(combined_score)

        print(f"AST Score (Average HTML & CSS): {ast_score}")
        print(f"STS Score: {sts_score}")
        print(f"TRS Score: {trs_score}")
        print(f"Combined Score: {combined_score}")
        print("-" * 50)

    avg_ast_score = sum(ast_scores) / len(ast_scores) if ast_scores else 0
    avg_sts_score = sum(sts_scores) / len(sts_scores) if sts_scores else 0
    avg_trs_score = sum(trs_scores) / len(trs_scores) if trs_scores else 0
    avg_combined_score = sum(combined_scores) / len(combined_scores) if combined_scores else 0

    print(f"Average AST Score: {avg_ast_score}")
    print(f"Average STS Score: {avg_sts_score}")
    print(f"Average TRS Score: {avg_trs_score}")
    print(f"Average Combined Score: {avg_combined_score}")

    return avg_ast_score, avg_sts_score, avg_trs_score, avg_combined_score

# Run RUBY evaluation on a sample of the test dataset
avg_ast_score, avg_sts_score, avg_trs_score, avg_combined_score = evaluate_ruby_sample(test_data, sample_size=5)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


AST Score (Average HTML & CSS): 0.5596184419713832
STS Score: 0.20967741935483872
TRS Score: 0.20967741935483872
Combined Score: 0.3263244268936869
--------------------------------------------------
AST Score (Average HTML & CSS): 0.5994623655913978
STS Score: 0.18613993973886844
TRS Score: 0.18613993973886844
Combined Score: 0.32391408168971153
--------------------------------------------------
AST Score (Average HTML & CSS): 0.6672320217096337
STS Score: 0.1575973832887303
TRS Score: 0.1575973832887303
Combined Score: 0.3274755960956981
--------------------------------------------------
AST Score (Average HTML & CSS): 1.08
STS Score: 0.4043010752688172
TRS Score: 0.4043010752688172
Combined Score: 0.6295340501792115
--------------------------------------------------
AST Score (Average HTML & CSS): 0.25476190476190474
STS Score: 0.023607012671411215
TRS Score: 0.023607012671411215
Combined Score: 0.10065864336824237
--------------------------------------------------
Average AST Score:

In [ ]:
import requests
import nltk
from bs4 import BeautifulSoup, Tag
from difflib import SequenceMatcher
import random
from nltk.tokenize import word_tokenize
import tinycss2
import re

nltk.download('punkt')

# Set up headers for OpenAI API
headers = {
    "Authorization": f"Bearer {openai_api_key}",  # Replace with your OpenAI API key
    "Content-Type": "application/json"
}

# Fine-tuned model ID
fine_tuned_model_id = "ft:gpt-4o-2024-08-06:personal::ANAuc42x"  # Replace with your actual fine-tuned model ID

# Function to calculate STS similarity
def calculate_sts_similarity(generated_code, reference_code):
    generated_tokens = word_tokenize(generated_code)
    reference_tokens = word_tokenize(reference_code)
    matcher = SequenceMatcher(None, generated_tokens, reference_tokens)
    return matcher.ratio()

# AST-based similarity for HTML using BeautifulSoup
def calculate_ast_similarity_html(generated_code, reference_code):
    generated_tree = BeautifulSoup(generated_code, "html.parser")
    reference_tree = BeautifulSoup(reference_code, "html.parser")

    def compare_trees(node1, node2):
        if not isinstance(node1, Tag) or not isinstance(node2, Tag):
            return 0
        if node1.name != node2.name:
            return 0

        score = 1
        if node1.attrs == node2.attrs:
            score += 1

        children1 = [child for child in node1.children if isinstance(child, Tag)]
        children2 = [child for child in node2.children if isinstance(child, Tag)]

        for child1, child2 in zip(children1, children2):
            score += compare_trees(child1, child2)
        return score

    try:
        ast_html_score = compare_trees(generated_tree, reference_tree) / max(len(generated_tree.find_all()), len(reference_tree.find_all()))
    except Exception as e:
        print(f"HTML AST comparison failed: {e}")
        ast_html_score = 0
    return ast_html_score

# AST-based similarity for CSS using TinyCSS2
def calculate_ast_similarity_css(generated_code, reference_code):
    generated_rules = tinycss2.parse_stylesheet(generated_code)
    reference_rules = tinycss2.parse_stylesheet(reference_code)

    def compare_css_rules(rule1, rule2):
        if rule1.type != rule2.type:
            return 0
        score = 1
        if hasattr(rule1, 'content') and hasattr(rule2, 'content') and rule1.content == rule2.content:
            score += 1
        return score

    score = 0
    try:
        for rule1, rule2 in zip(generated_rules, reference_rules):
            score += compare_css_rules(rule1, rule2)
        ast_css_score = score / max(len(generated_rules), len(reference_rules))
    except Exception as e:
        print(f"CSS AST comparison failed: {e}")
        ast_css_score = 0
    return ast_css_score

# TRS-based similarity
def calculate_trs_similarity(generated_code, reference_code):
    generated_tokens = word_tokenize(generated_code)
    reference_tokens = word_tokenize(reference_code)
    matcher = SequenceMatcher(None, generated_tokens, reference_tokens)
    return matcher.ratio()

# RUBY metric
def ruby_metric(generated_code, reference_code):
    ast_html_score = calculate_ast_similarity_html(generated_code, reference_code)
    ast_css_score = calculate_ast_similarity_css(generated_code, reference_code)
    ast_score = (ast_html_score + ast_css_score) / 2 if (ast_html_score or ast_css_score) else 0
    sts_score = calculate_sts_similarity(generated_code, reference_code)
    trs_score = calculate_trs_similarity(generated_code, reference_code)
    combined_score = (ast_score + sts_score + trs_score) / 3
    return ast_score, sts_score, trs_score, combined_score

# Function to evaluate Pass@1
def extract_relevant_criteria(prompt):
    criteria = {
        "has_header_container": "header" in prompt.lower(),
        "has_navigation_menu": "navigation" in prompt.lower() or "menu" in prompt.lower(),
        "includes_css_styling": "css" in prompt.lower(),
    }
    return {k: v for k, v in criteria.items() if v}

def evaluate_pass_at_1(generated_code, relevant_criteria):
    soup = BeautifulSoup(generated_code, 'html.parser')
    results = {}
    if relevant_criteria.get("has_header_container"):
        results["has_header_container"] = bool(soup.find('header') or soup.find('div', {'class': re.compile('header', re.IGNORECASE)}))
    if relevant_criteria.get("has_navigation_menu"):
        results["has_navigation_menu"] = bool(soup.find('nav') or soup.find_all('a', href=True))
    if relevant_criteria.get("includes_css_styling"):
        results["includes_css_styling"] = '<style>' in generated_code or 'style=' in generated_code
    pass_at_1 = all(results.values())
    return pass_at_1, results

# Evaluation function
def evaluate_model(test_data, sample_size=5):
    sample_data = random.sample(test_data, sample_size)
    ast_scores, sts_scores, trs_scores, combined_scores = [], [], [], []
    pass_count = 0

    for item in sample_data:
        prompt = item['messages'][1]['content']
        reference_code = item['messages'][2]['content']
        relevant_criteria = extract_relevant_criteria(prompt)

        sample_conversation = [
            {"role": "system", "content": "Provide structured HTML and CSS for UI components of a web template."},
            {"role": "user", "content": prompt}
        ]

        response = requests.post(
            "https://api.openai.com/v1/chat/completions",
            headers=headers,
            json={
                "model": fine_tuned_model_id,
                "messages": sample_conversation
            }
        )

        if response.status_code == 200:
            generated_code = response.json()["choices"][0]["message"]["content"].strip()
        else:
            print("Error:", response.status_code, response.text)
            generated_code = ""

        # Calculate RUBY metric
        ast_score, sts_score, trs_score, combined_score = ruby_metric(generated_code, reference_code)
        ast_scores.append(ast_score)
        sts_scores.append(sts_score)
        trs_scores.append(trs_score)
        combined_scores.append(combined_score)

        # Pass@1 evaluation
        pass_at_1, criteria_results = evaluate_pass_at_1(generated_code, relevant_criteria)
        if pass_at_1:
            pass_count += 1

        # Print individual sample results
        print(f"Prompt: {prompt}")
        print(f"AST Score (Average HTML & CSS): {ast_score}")
        print(f"STS Score: {sts_score}")
        print(f"TRS Score: {trs_score}")
        print(f"Combined Score: {combined_score}")
        print("Pass@1:", "Passed" if pass_at_1 else "Failed")
        print("Criteria Results:", criteria_results)
        print("-" * 50)

    # Calculate and print averages
    avg_ast_score = sum(ast_scores) / len(ast_scores) if ast_scores else 0
    avg_sts_score = sum(sts_scores) / len(sts_scores) if sts_scores else 0
    avg_trs_score = sum(trs_scores) / len(trs_scores) if trs_scores else 0
    avg_combined_score = sum(combined_scores) / len(combined_scores) if combined_scores else 0
    pass_at_1_rate = (pass_count / len(sample_data)) * 100 if sample_data else 0

    print(f"Average AST Score: {avg_ast_score}")
    print(f"Average STS Score: {avg_sts_score}")
    print(f"Average TRS Score: {avg_trs_score}")
    print(f"Average Combined Score: {avg_combined_score}")
    print(f"Pass@1 Rate: {pass_at_1_rate}%")

# Run evaluation
evaluate_model(test_data, sample_size=6)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Prompt: Create a header of an eCommerce website under the "Products" category, featuring a modern and clean look with navigation links (Home, Recent Products, Brands, Contact, About), a search bar, cart icon, and branding. Include all HTML and CSS components to ensure responsiveness and intuitive UI/UX, optimizing for user engagement and seamless shopping experience.
AST Score (Average HTML & CSS): 0.703861097620204
STS Score: 0.11894969946219551
TRS Score: 0.11894969946219551
Combined Score: 0.31392016551486496
Pass@1: Failed
Criteria Results: {'has_header_container': False, 'has_navigation_menu': False, 'includes_css_styling': True}
--------------------------------------------------
Prompt: Create a header of a fashion e-commerce website, featuring a clean, modern look. Include an announcement bar, logo, navigation menu for New Arrivals, Swimwear, Collections, About Us, and Demos, a search icon, account link, and cart icon. Use responsive design, bold typography, intuitive layout, an

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import json
import nltk
import random

nltk.download('punkt')

# Set up headers for OpenAI API
headers = {
    "Authorization": f"Bearer YOUR_OPENAI_API_KEY",  # Replace with your OpenAI API key
    "Content-Type": "application/json"
}

# Fine-tuned model ID
fine_tuned_model_id = "ft:gpt-4o-2024-08-06:personal::ANAuc42x"  # Replace with your actual fine-tuned model ID

# Function to extract relevant criteria from prompt
def extract_relevant_criteria(prompt):
    relevant_criteria = {
        "has_header_container": "header" in prompt.lower(),
        "has_basic_html_structure": "html structure" in prompt.lower(),
        "has_navigation_menu": "navigation" in prompt.lower() or "menu" in prompt.lower(),
        "has_responsive_design": "responsive" in prompt.lower(),
        "includes_css_styling": "css" in prompt.lower() or "style" in prompt.lower(),
        "has_call_to_action": "call-to-action" in prompt.lower() or "button" in prompt.lower(),
        "uses_semantic_tags": "semantic" in prompt.lower() or "structure" in prompt.lower(),
        "no_syntax_errors": True  # Assume BeautifulSoup can parse without errors
    }
    return {k: v for k, v in relevant_criteria.items() if v}

# Success criteria definition for Pass@1 evaluation
def evaluate_pass_at_1(generated_code, relevant_criteria):
    soup = BeautifulSoup(generated_code, 'html.parser')

    # Generalized Criteria checks based on relevant criteria
    results = {}
    if relevant_criteria.get("has_header_container"):
        results["has_header_container"] = bool(soup.find('header') or soup.find('div', {'class': re.compile('header', re.IGNORECASE)}))
    if relevant_criteria.get("has_basic_html_structure"):
        results["has_basic_html_structure"] = all([
            bool(soup.find('html')),
            bool(soup.find('head')),
            bool(soup.find('body'))
        ])
    if relevant_criteria.get("has_navigation_menu"):
        results["has_navigation_menu"] = bool(soup.find('nav') or soup.find('ul') or soup.find_all('a', href=True))
    if relevant_criteria.get("has_responsive_design"):
        results["has_responsive_design"] = '@media' in generated_code or bool(soup.find('meta', {'name': 'viewport'}))
    if relevant_criteria.get("includes_css_styling"):
        results["includes_css_styling"] = '<style>' in generated_code or 'stylesheet' in generated_code or 'style=' in generated_code
    if relevant_criteria.get("has_call_to_action"):
        results["has_call_to_action"] = bool(soup.find('a', {'class': re.compile('button|cta', re.IGNORECASE)}) or soup.find('button'))
    if relevant_criteria.get("uses_semantic_tags"):
        results["uses_semantic_tags"] = any(tag.name in ['header', 'nav', 'section'] for tag in soup.find_all())
    if relevant_criteria.get("no_syntax_errors"):
        results["no_syntax_errors"] = True  # BeautifulSoup parsing indicates no syntax errors

    # Check if all relevant criteria are met for Pass@1
    pass_at_1 = all(results.values())
    return pass_at_1, results

def evaluate_model(test_data, sample_size=5):
    # Limit to sample_size samples for evaluation
    sample_data = random.sample(test_data, sample_size)

    pass_count = 0
    detailed_logs = []

    for item in sample_data:
        prompt = item['messages'][1]['content']
        reference_code = item['messages'][2]['content']

        # Extract relevant criteria from the prompt
        relevant_criteria = extract_relevant_criteria(prompt)

        # Prepare request payload for OpenAI API call
        sample_conversation = [
            {"role": "system", "content": "Provide structured HTML and CSS for UI components of a web template."},
            {"role": "user", "content": prompt}
        ]

        response = requests.post(
            "https://api.openai.com/v1/chat/completions",
            headers=headers,
            json={
                "model": fine_tuned_model_id,
                "messages": sample_conversation
            }
        )

        if response.status_code == 200:
            generated_code = response.json()["choices"][0]["message"]["content"].strip()
        else:
            print("Error:", response.status_code, response.text)
            generated_code = ""

        # Evaluate Pass@1 criteria and log results
        pass_at_1, criteria_results = evaluate_pass_at_1(generated_code, relevant_criteria)

        # Count the passes
        if pass_at_1:
            pass_count += 1

        # Log details for this example
        detailed_logs.append({
            "prompt": prompt,
            "generated_code": generated_code,
            "reference_code": reference_code,
            "pass_at_1": "Passed" if pass_at_1 else "Failed",
            "criteria_results": criteria_results  # Detailed criteria outcomes
        })

        # Print each result
        print(f"Prompt: {prompt}")
        print(f"Generated Code:\n{generated_code}")
        print(f"Reference Code:\n{reference_code}")
        print("Pass@1:", "Passed" if pass_at_1 else "Failed")
        print("Relevant Criteria Evaluated:", relevant_criteria)
        print("Detailed Criteria Results:")
        for criterion, passed in criteria_results.items():
            status = "Pass" if passed else "Fail"
            print(f"  {criterion}: {status}")
        print("-" * 50)

    # Calculate Pass@1 rate
    pass_at_1_rate = (pass_count / len(sample_data)) * 100 if sample_data else 0
    print(f"Pass@1 Rate for {sample_size} samples: {pass_at_1_rate:.2f}%")

    # Log the overall summary
    overall_log = {
        "pass_at_1_rate": pass_at_1_rate,
        "details": detailed_logs
    }

    return pass_at_1_rate, overall_log

# Run the evaluation on test_data with a sample size of 5
pass_at_1_rate, evaluation_log = evaluate_model(test_data, sample_size=6)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Error: 401 {
    "error": {
        "message": "Incorrect API key provided: YOUR_OPE*******_KEY. You can find your API key at https://platform.openai.com/account/api-keys.",
        "type": "invalid_request_error",
        "param": null,
        "code": "invalid_api_key"
    }
}

Prompt: Create a header of a Professional Portfolio Website focusing on a creative and vibrant layout. Utilize HTML for structured sections like About Me, Services, and buttons such as Hire Me and Download CV. Style with CSS to emphasize a modern UI, incorporating color gradients, rounded buttons, and clear typography, enhancing readability and user interaction while ensuring visual consistency.
Generated Code:

Reference Code:
<html>
<head>
<style>
.frame-work-1 {
  	position: absolute;
  	top: 156px;
  	left: 757px;
  	width: 617px;
  	height: 576.4px;
  	object-fit: cover;
}
.group-child {
  	position: absolute;
  	top: 0px;
  	left: 0px;
  	border-radius: 50%;
  	background-color: #f9c61b;
  	border: 4px s